In [1]:
import facerec

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [4]:
import numpy as np
import pandas as pd
import cv2

In [5]:
facerec.r.ping()

True

### Extract data from database

In [8]:
name='academy:register'
retrive_dict = facerec.r.hgetall(name)
retrive_series = facerec.pd.Series(retrive_dict)
retrive_series = retrive_series.apply(lambda x: np.frombuffer(x,dtype=np.float32))
index = retrive_series.index
index = list(map(lambda x: x.decode(),index))
retrive_series.index = index
retrive_df = retrive_series.to_frame().reset_index()
retrive_df.columns = ['name_role','facial_features']
retrive_df[['Name','Role']]=retrive_df['name_role'].apply(lambda x: x.split('@')).apply(pd.Series)
retrive_df

,name_role,facial_features,Name,Role
0,Angelina Jolie@Student,"[0.43242025, 1.0748498, -0.12862365, -1.833291...",Angelina Jolie,Student
1,Pratik Raut@Student,"[-1.2463742, -0.46773538, 0.40567505, 2.755777...",Pratik Raut,Student
2,Barack Obama@Teacher,"[0.9485809, -0.5584051, 0.094820715, -1.279659...",Barack Obama,Teacher
3,Chris Evans@Student,"[0.7201714, -1.4684148, -0.7781253, -1.4601073...",Chris Evans,Student
4,Morgan Freeman@Teacher,"[0.48535094, -0.6192554, -0.4069932, 1.0687788...",Morgan Freeman,Teacher
5,Scarlett Johansson@Student,"[0.19131956, -0.48843804, -1.690233, 0.4423967...",Scarlett Johansson,Student


### Step 2 Get Real - Time Prediction

In [9]:
cap = cv2.VideoCapture(0) #default , 1 for external camera

while True:
    ret , frame = cap.read()
    if ret == False:
        break
    
    pred_frame = facerec.face_prediction(frame,retrive_df,'facial_features',['Name','Role'],thresh=0.5)
    cv2.imshow('Frame',frame)
    cv2.imshow('Prediction',pred_frame)

    if cv2.waitKey(1) == 27: # if I press esc button this condition will get triggered
        break

cv2.destroyAllWindows()
cap.release()